In [85]:
import pandas as pd
import numpy as np
import random
import copy
import math

In [86]:
actions_map = {
    0: 'take 1 coin',
    1: 'coup',
    2: 'take 2 coins',
    3: 'take 3 coins',
    4: 'steal 2 coins',
    5: 'assassinate',
    6: 'exchange',
    7: 'challenge',
    8: 'block foreign aid',
    9: 'block stealing',
    10: 'block assassination'
}

In [87]:
class Action:
    def __init__(self, name, challengeable, response_card, response_action, 
                 p1_net_coins, p2_net_coins, p1_net_cards, p2_net_cards, vector):
        self.name = name
        self.challengeable = challengeable
        self.response_card = response_card
        self.response_action = response_action
        self.p1_net_coins = p1_net_coins
        self.p2_net_coins = p2_net_coins
        self.p1_net_cards = p1_net_cards
        self.p2_net_cards = p2_net_cards
#         self.base_utility = base_utility
#         self.p_bluff = p_bluff
        self.vector = vector
    
    def update_responses(self, response_card, response_action):
        self.response_card = response_card
        self.response_action = response_action
        

In [88]:
take_1 = Action(actions_map[0], False, None, None, 1, 0, 0, 0, [0])

coup = Action(actions_map[1], False, None, None, -7, 0, 0, -1, [1])

take_2 = Action(actions_map[2], True, 'Duke', actions_map[8], 2, 0, 0, 0, [2])

take_3 = Action(actions_map[3], True, None, actions_map[7], 3, 0, 0, 0, [3])

steal_2 = Action(actions_map[4], True, ['Captain', 'Ambassador'], actions_map[9], 2, -2, 0, 0,[4])

assassinate = Action(actions_map[5], True, 'Contessa', actions_map[10], -3, 0, 0, -1, [5])

exchange = Action(actions_map[6], True, None, actions_map[7], 0, 0, 0, 0,[6])

# challenge = Action(actions_map[7], False, None, None, 0, 0, -1, -1, 1, 0)

block_take_2 = Action(actions_map[8], True, None, actions_map[7], 0, -2, 0, 0, [7])

block_steal = Action(actions_map[9], True, None, actions_map[7], 2, -2, 0, 0, [8])

block_assassination = Action(actions_map[10], True, None, actions_map[7], 0, 0, 1, 0, [9])

# challenge = 

actions = {
    0: take_1,
    1: coup,
    2: take_2,
    3: take_3,
    4: steal_2,
    5: assassinate,
    6: exchange,
    7: block_take_2,
    8: block_steal,
    9: block_assassination
}

take_2.response_action = actions[7]
steal_2.response_action = actions[8]
assassinate.response_action = actions[9]

influences = {
    'Duke': [take_3, block_take_2, take_1, coup],
    'Captain': [steal_2, block_steal, take_2, take_1, coup],
    'Assassin': [assassinate, take_2, take_1, coup],
    'Contessa': [take_2, block_assassination, take_1, coup],
    'Ambassador': [exchange, block_steal, take_2, take_1, coup]
    }


influences_reverse = {
    take_1: ['Duke', 'Captain', 'Assassin', 'Contessa', 'Ambassador'],
    coup: ['Duke', 'Captain', 'Assassin', 'Contessa', 'Ambassador'],
    take_2: ['Captain', 'Assassin', 'Contessa', 'Ambassador'],
    take_3: ['Duke'],
    steal_2: ['Captain'],
    assassinate: ['Assassin'],
    exchange: ['Ambassador'],
    block_take_2: ['Duke'],
    block_steal: ['Captain','Ambassador'],
    block_assassination: ['Contessa']
}

In [89]:
class Bot:
    def __init__(self, bluff_degree, cards, num_coins, hostility, name):
        self.bluff_degree = bluff_degree
        self.cards = cards
        self.num_coins = num_coins
        self.hostility = hostility
        self.kb = None
        self.name = name
        
    def num_coins_adj(self, n):
        self.num_coins += n
    
    def cards_adj(self, card):
        self.cards.remove(card)
        
    def build_kb(self, kb):
        self.kb = kb
        
    def update_kb(self, influences, influence, action, kb, base_or_response, i, ):
        alpha = utility(influences, action, )
        for _ in kb:
            likelihood = math.exp(-alpha/5)
            kb *= likelihood
        self.kb[i] = [num / sum(kb) for num in kb]
        
        

In [90]:
bag = ['Duke', 'Captain', 'Assassin', 'Contessa', 'Ambassador'] * 3
random.shuffle(bag)

bots = []
bluff_degree = 0

for i in range(4):
    cards = random.sample(bag, 2)
    for card in cards:
        bag.remove(card)
#     kb = []
    
    bots.append(Bot(bluff_degree, cards, 2, None, f'{i}'))
    bluff_degree += 0.25
    
# Bot 0
kb_j = []
m = 0
for influence in influences.keys():
    if bots[0].cards[0] == influence: 
        m += 1
    if bots[0].cards[1] == influence:
        m += 1
    kb_j.append(1 - (math.comb(13-(3-m), 2) / math.comb(13, 2)))
    m = 0
kb_j = [num / sum(kb_j) for num in kb_j]
kb = {
    bots[1]: copy.deepcopy(kb_j),
    bots[2]: copy.deepcopy(kb_j),
    bots[3]: copy.deepcopy(kb_j),
}
bots[0].build_kb(kb)



# Bot 1
kb_j = []
# m = 0
for influence in influences.keys():
    if bots[1].cards[0] == influence: 
        m += 1
    if bots[1].cards[1] == influence:
        m += 1
    kb_j.append(1 - (math.comb(13-(3-m), 2) / math.comb(13, 2)))
    m = 0
kb_j = [num / sum(kb_j) for num in kb_j]
kb = {
    bots[0]: copy.deepcopy(kb_j),
    bots[2]: copy.deepcopy(kb_j),
    bots[3]: copy.deepcopy(kb_j),
}
bots[1].build_kb(kb)



# Bot 2
kb_j = []
# m = 0
for influence in influences.keys():
    if bots[2].cards[0] == influence: 
        m += 1
    if bots[2].cards[1] == influence:
        m += 1
    kb_j.append(1 - (math.comb(13-(3-m), 2) / math.comb(13, 2)))
    m = 0
kb_j = [num / sum(kb_j) for num in kb_j]
kb = {
    bots[0]: copy.deepcopy(kb_j),
    bots[1]: copy.deepcopy(kb_j),
    bots[3]: copy.deepcopy(kb_j),
}
bots[2].build_kb(kb)



# Bot 3
kb_j = []
# m = 0
for influence in influences.keys():
    if bots[3].cards[0] == influence: 
        m += 1
    if bots[3].cards[1] == influence:
        m += 1
    kb_j.append(1 - (math.comb(13-(3-m), 2) / math.comb(13, 2)))
    m = 0
kb_j = [num / sum(kb_j) for num in kb_j]
kb = {
    bots[0]: copy.deepcopy(kb_j),
    bots[1]: copy.deepcopy(kb_j),
    bots[2]: copy.deepcopy(kb_j),
}
bots[3].build_kb(kb)






for bot in bots:
    print(bot.cards)
print(bots[0].name)

['Ambassador', 'Contessa']
['Ambassador', 'Captain']
['Contessa', 'Assassin']
['Assassin', 'Duke']
0


In [91]:
def action_selection(i, bots, actions_vector, actions):
    
    action = random.choice(actions_vector)
    
    target = None
    
    if actions[action].response_action is not None and actions[action].response_action != 'challenge':  # has target?
        
        targets = bots[:i] + bots[i+1:]
        for j in targets:
            if j.num_coins < actions[action].p2_net_coins * (-1):
                targets.remove(j)
        
        target = random.choice(targets)
        target = bots.index(target)
        
    return [action, target]


def reaction_selection(i, bots, target, response_action):
    
    return random.choice([0, 1])


def challenge_selection(i, bots, target, action):
    
    return random.choice([0, 1])


def card_selection(cards):
    
    c = random.choice(cards)
    
    c = bot.cards.index(c)
    
    return 

In [92]:
def perform_action(bot, target, action):
    
    if target is not None:
        
        target.num_coins += action.p2_net_coins
        
        if action.p2_net_cards < 0:
            
            card = card_selection(target.cards)
            
            x = target.cards[card]
                        
            discard_pile.append(x)

            target.cards.remove(x)
    
    bot.num_coins += action.p1_net_coins
    
    if action == exchange:
        
        card = card_selection(bot.cards)
        
        x = bot.cards[card]
            
        c = random.sample(bag, 2)
        
        arr = x + c
        
        final_choice = card_selection(arr)
        
        final_choice = arr[final_choice]
        
        arr.remove(final_choice)
        
        for i in arr:
            bag.insert(i)
        
        random.shuffle(bag)
        bot.cards.insert(final_choice)
        bot.cards.remove(x)
        
        

In [93]:
# Base Game Loop

discard_pile = []

while len(bots) > 1:
    
    for i in range(len(bots)):
        
        action_stack = []
        
        action_vector = [0,1,2,3,4,5,6]
        for j in action_vector:
            if actions[j].p1_net_coins * (-1) > bots[i].num_coins:
                action_vector.remove(j)
        
        action_selection_output = action_selection(i, bots, action_vector, actions)
        
        action = action_selection_output[0]
        action = actions[action]
#         print(action_selection_output[1])

        print(f'bot {bots[i].name} is performing action {action.name}')
        if target is not None:
            print(f'target is {target.name}')
        
        try:
            target = bots[action_selection_output[1]]
        except TypeError:
            print("no target")
        
        action_stack.append(action)
        
        
        if action.response_action is not None and action.response_action != 'challenge':  # is blockable?
            
            response = reaction_selection(bots[i], bots, target, action.response_action)
            
            if response == 1:
                
                action_stack.append(action.response_action)
                
                print(f'bot {target.name} is performing action {action.response_action} against bot {bots[i].name}')
                
            else:
                print(f'target will not block')
            
        if action_stack[-1].response_action == 'challenge':  # is challengeable?
            
            response = challenge_selection(i, bots, target, action)
            
            if response == 1:
                
                action_stack.append('challenge')
                
                if len(action_stack) == 3:
                    print(f'bot {bots[i].name} challenges bot {target.name}')
                    
                else: 
                    print(f'bot {target.name} challenges bot {bots[i].name}')
            
            else:
                
                print('no challenge')
        
        while len(action_stack) != 0:
            
            a = action_stack.pop()
            
            if a == 'challenge':
                
                if len(action_stack) > 1:
                    
                    if influences_reverse[action_stack[-1]] in target.cards:
                        
                        card = card_selection(bots[i].cards)
                        
                        x = bots[i].cards[card]
                        
                        discard_pile.append(x)
                        
                        bots[i].cards.remove(x)
                        
                        if len(bots[i].cards) == 0:
                            
                            bots.remove(bots[i])
                        
                        target.cards.remove(influences_reverse[action_stack[-1]])
                        bag.insert(influences_reverse[action_stack[-1]])
                        random.shuffle(bag)
                        c = random.sample(bag, 1)
                        bag.remove(c)
                        target.cards.insert(c)
                        
                        action_stack.pop()
                        action_stack.pop()
                        
                    else:
                        
                        card = card_selection(target.cards)
                        
                        x = target.cards[card]
                        
                        discard_pile.append(x)
                        
                        target.cards.remove(x)
                        
                        if len(target.cards) == 0:
                            
                            bots.remove(target)
                        
                        action_stack.pop()
                
                else:
                    
                    if influences_reverse[action_stack[-1]] in bots[i].cards:
                        
                        card = card_selection(target.cards)
                        
                        x = target.cards[card]
                        
                        discard_pile.append(x)
                        
                        target.cards.remove(x)
                        
                        if len(target.cards) == 0:
                            
                            bots.remove(target)
                        
                        bots[i].cards.remove(influences_reverse[action_stack[-1]])
                        bag.insert(influences_reverse[action_stack[-1]])
                        random.shuffle(bag)
                        c = random.sample(bag, 1)
                        bag.remove(c)
                        bots[i].cards.insert(c)
                        
                    else:
                        
                        card = card_selection(bots[i].cards)
                        
                        x = target.cards[card]
                        
                        discard_pile.append(x)
                        
                        bots[i].cards.remove(x)
                        
                        if len(bots[i].cards) == 0:
                            
                            bots.remove(bots[i])
                        
                        action_stack.pop()
                        
            else:
                
                if len(action_stack) == 1:
                    
                    perform_action(target, bots[i], a)
                    if len(bots[i].cards) == 0:
                        bots.remove(bots[i])
                
                else:
                    
                    perform_action(bots[i], target, a)
                    if len(target.cards) == 0:
                        bots.remove(target)
                        
                        
    print(f'bot {bots[i].name} has {bots[i].num_coins} coins.')
    print(f'bot {target.name} has {target.num_coins} coins.')

    print(f'bot {bots[i].name} has {len(bots[i].cards)} cards.')
    print(f'bot {target.name} has {len(target.cards)} cards.')
        
    print(discard_pile)
        
print(f'bot {bots[0].name} wins!')
                

bot 0 is performing action steal 2 coins
target is 0
bot 3 is performing action <__main__.Action object at 0x1083c60d0> against bot 0
bot 0 challenges bot 3


TypeError: list indices must be integers or slices, not NoneType